In [1]:
import pandas as pd
from prophet import Prophet
from IPython.display import clear_output
import os
import logging


logging.getLogger('prophet').setLevel(logging.WARNING) # Removes te 'INFO' warnings from Prophet

Importing plotly failed. Interactive plots will not work.


In [2]:
# Read the data. There are 4 options. All time, last 5 years, last year, and last 6 months.

data_import = pd.read_csv("VOOmax.csv", parse_dates=["Date"])
#data_import = pd.read_csv("VOO5y.csv", parse_dates=["Date"])
#data_import = pd.read_csv("VOO1y.csv", parse_dates=["Date"])
#data_import = pd.read_csv("VOO6m.csv", parse_dates=["Date"])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720139,8600
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747871,33750
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.731827,59400
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.940575,9250
...,...,...,...,...,...,...,...
2898,2022-03-15,385.859985,392.369995,384.630005,391.750000,391.750000,5739400
2899,2022-03-16,395.500000,400.549988,390.549988,400.510010,400.510010,7407300
2900,2022-03-17,398.640015,405.559998,398.299988,405.410004,405.410004,5177500
2901,2022-03-18,403.980011,410.279999,403.239990,409.890015,409.890015,5873400


In [3]:
# Add a column of the day names

day_mapper = {0: "Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
data_import["DayOfWeek"] = data_import["Date"].map(lambda x: day_mapper[x.dayofweek])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500,Thursday
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720139,8600,Friday
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747871,33750,Monday
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.731827,59400,Tuesday
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.940575,9250,Wednesday
...,...,...,...,...,...,...,...,...
2898,2022-03-15,385.859985,392.369995,384.630005,391.750000,391.750000,5739400,Tuesday
2899,2022-03-16,395.500000,400.549988,390.549988,400.510010,400.510010,7407300,Wednesday
2900,2022-03-17,398.640015,405.559998,398.299988,405.410004,405.410004,5177500,Thursday
2901,2022-03-18,403.980011,410.279999,403.239990,409.890015,409.890015,5873400,Friday


# Method 1: Average Dip per Day

In [4]:
# Take the previous value and then subtract it from the current value, check the difference

data_import["diff_from_previous_day"] = (data_import["Open"].diff() / data_import["Open"] + 1)
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500,Thursday,NaN
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720139,8600,Friday,0.991935
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747871,33750,Monday,1.012432
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.731827,59400,Tuesday,0.998833
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.940575,9250,Wednesday,0.997856
...,...,...,...,...,...,...,...,...,...
2898,2022-03-15,385.859985,392.369995,384.630005,391.750000,391.750000,5739400,Tuesday,0.996864
2899,2022-03-16,395.500000,400.549988,390.549988,400.510010,400.510010,7407300,Wednesday,1.024374
2900,2022-03-17,398.640015,405.559998,398.299988,405.410004,405.410004,5177500,Thursday,1.007877
2901,2022-03-18,403.980011,410.279999,403.239990,409.890015,409.890015,5873400,Friday,1.013218


In [5]:
# Compare the different days to see which one has the biggest dip from the previous day

data_import.groupby("DayOfWeek")["diff_from_previous_day"].mean()

DayOfWeek
Friday       1.000698
Monday       0.999633
Thursday     0.999921
Tuesday      1.000969
Wednesday    1.000816
Name: diff_from_previous_day, dtype: float64

# Method 2: Backtester Method

Tracks the number of shares that I own based on the number of shares that I bought when and what happened to the price on any given day

In [6]:
# Create a table of every day between the minimum date and maximum date to deal with holidays
dates = pd.date_range(start=data_import["Date"].min(), end=data_import["Date"].max())

# Add the name of the day of the week
date_table = pd.DataFrame(data={"Calendar Date":dates})
date_table["Weekday"] = date_table["Calendar Date"].map(lambda x: day_mapper[x.dayofweek])
date_table

,Calendar Date,Weekday
0,2010-09-09,Thursday
1,2010-09-10,Friday
2,2010-09-11,Saturday
3,2010-09-12,Sunday
4,2010-09-13,Monday
...,...,...
4204,2022-03-14,Monday
4205,2022-03-15,Tuesday
4206,2022-03-16,Wednesday
4207,2022-03-17,Thursday


In [7]:
# Create a table of all dates and remove the weekends
full_calendar = pd.merge(left= date_table, right= data_import, how='left', left_on='Calendar Date', right_on='Date')
full_calendar.rename(columns={'Date':'Trading Day'}, inplace=True)
full_calendar = full_calendar[~full_calendar["Weekday"].isin(["Saturday", "Sunday"])]
full_calendar


,Calendar Date,Weekday,Trading Day,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2010-09-09,Thursday,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500.0,Thursday,NaN
1,2010-09-10,Friday,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720139,8600.0,Friday,0.991935
4,2010-09-13,Monday,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747871,33750.0,Monday,1.012432
5,2010-09-14,Tuesday,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.731827,59400.0,Tuesday,0.998833
6,2010-09-15,Wednesday,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.940575,9250.0,Wednesday,0.997856
...,...,...,...,...,...,...,...,...,...,...,...
4205,2022-03-15,Tuesday,2022-03-15,385.859985,392.369995,384.630005,391.750000,391.750000,5739400.0,Tuesday,0.996864
4206,2022-03-16,Wednesday,2022-03-16,395.500000,400.549988,390.549988,400.510010,400.510010,7407300.0,Wednesday,1.024374
4207,2022-03-17,Thursday,2022-03-17,398.640015,405.559998,398.299988,405.410004,405.410004,5177500.0,Thursday,1.007877
4208,2022-03-18,Friday,2022-03-18,403.980011,410.279999,403.239990,409.890015,409.890015,5873400.0,Friday,1.013218


In [8]:
# For any closed market days, take the next available day's opening value as the price I'd buy the stock at
full_calendar = full_calendar.bfill(axis = 'rows').reset_index(drop=True)
full_calendar.to_clipboard()


In [9]:
# Check number of each Weekday
full_calendar['Weekday'].value_counts()


Friday       603
Thursday     602
Monday       601
Tuesday      601
Wednesday    601
Name: Weekday, dtype: int64

In [10]:
# For each Weekday from 'Monday' to 'Friday' we need to create a function to backtest as if we invested from the beginning
def day_backtester(day, amount_to_invest, data):
    temp_data = data[data['Weekday']==day]
    temp_data['Shares Owned'] = amount_to_invest / temp_data['Open']
    final_price = temp_data['Open'].iloc[-1]
    final_amount = temp_data['Shares Owned'].sum() * final_price
    formatted_final_amount = "${:,.2f}".format(final_amount)
    return formatted_final_amount
    


In [11]:
for i in full_calendar['Weekday'].unique():
    print(i, day_backtester(i, 750, full_calendar))

Thursday $952,178.96
Friday $965,069.64
Monday $923,270.30
Tuesday $919,824.30
Wednesday $941,879.72


C:\Users\RJ\AppData\Local\Temp\ipykernel_11708\2601197036.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['Shares Owned'] = amount_to_invest / temp_data['Open']
C:\Users\RJ\AppData\Local\Temp\ipykernel_11708\2601197036.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['Shares Owned'] = amount_to_invest / temp_data['Open']
C:\Users\RJ\AppData\Local\Temp\ipykernel_11708\2601197036.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [12]:
# For each weekday, see the profit if you sell the next available day
def sell_next_day_backtester(day, amount_to_invest, data):
    temp_data = data[data['Weekday']==day]
    temp_data['Profit'] = temp_data['diff_from_previous_day'] * amount_to_invest - amount_to_invest
    final_profit = temp_data['Profit'].sum()
    formatted_final_profit = "${:,.2f}".format(final_profit)
    return formatted_final_profit
    

In [13]:
for i in full_calendar['Weekday'].unique():
    print(i, sell_next_day_backtester(i, 10000, full_calendar))

Thursday $-629.79
Friday $4,052.30
Monday $-713.26
Tuesday $5,913.18
Wednesday $4,525.72


C:\Users\RJ\AppData\Local\Temp\ipykernel_11708\1011213871.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['Profit'] = temp_data['diff_from_previous_day'] * amount_to_invest - amount_to_invest
C:\Users\RJ\AppData\Local\Temp\ipykernel_11708\1011213871.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['Profit'] = temp_data['diff_from_previous_day'] * amount_to_invest - amount_to_invest
C:\Users\RJ\AppData\Local\Temp\ipykernel_11708\1011213871.py:4: SettingWithCopyWarning: 
A value is 